# An Introduction to the Gaussian Distribution Class

This tutorial illustrates how to use `probpipe.distributions.gaussian`, proppipe's implementation of a multivariate
Gaussian distribution.


In [24]:
import numpy as np

from probpipe.distributions.gaussian import Gaussian
from probpipe.linalg.linop import DiagonalLinOp, TriangularLinOp, CholeskyLinOp

## The Basics
We begin by constructing a bivariate Gaussian distribution.

In [2]:
m = np.array([0, 1])
C = np.array([[1.0, 0.8], [0.8, 1.0]])
x = Gaussian(m, C)

print(x)

The `sample()` and `log_density()` methods are used to draw samples and evaluate the log probability
density function, respectively. A set of `n` samples is returned as a `(n, d)` array, with `d` the 
dimension of the distribution. The `log_density()` method is vectorized to compute the log density
over a batch of input points, stored as an array of shape `(n, d)` or `(d,)`, the latter corresponding
to a single point. Notice that a single sample is returned as an array of shape `(1, d)` for consistency
with the batch convention. Other methods such as `density()` and `cdf()` follow similar batch conventions.

In [ ]:
# Draw samples
samp_batch = x.sample(5)
single_sample = x.sample()

# Evaluate log density
log_dens_batch = x.log_density(samp_batch)
log_dens_single_sample = x.log_density(single_sample)

print("Batch of samples:")
print(samp_batch)

print("\nSingle sample:")
print(single_sample)

print("\nLog density at batch of samples:")
print(log_dens_batch)

print("\nLog density at single sample:")
print(log_dens_single_sample)


Batch of samples:
[[-0.73274273  0.37103335]
 [ 0.47547456  2.21613794]
 [-1.03563178  0.88648901]
 [ 2.31598751  2.5603977 ]
 [ 0.40415759  1.96711464]]

Single sample:
[[0.09218901 0.63250623]]

Log density at batch of samples:
[-1.59804835 -2.41021715 -2.57334172 -4.12769117 -1.98436718]

Log density at single sample:
[-1.60171353]


The mean and covariance of the Gaussian distribution can be accessed as follows. These properties
are also defined for any subclass of `RealVectorDistribution`.

In [ ]:
print(x.mean)
print(x.cov)

[0 1]
DenseLinOp(shape=(2, 2), dtype=float64)


Observe that the mean is a flat array of length `d`, as expected. On the other hand, the covariance is
a `DenseLinOp` instance. A linear operator is simply an abstract representation of a matrix. A dense 
linear operator actually stores the full matrix as a dense array. The concrete array representation of
any linear operator can be obtained via the `to_dense()` method. Note that the `Gaussian` constructor
accepted the array representation of the covariance, but this was wrapped as a `DenseLinOp` under 
the hood.

In [8]:
cov_op = x.cov
cov_op.to_dense()

array([[1. , 0.8],
       [0.8, 1. ]])

## Structured Covariance Operators

The benefit of representing the covariance as a linear operator is that we can pass linear 
operators representing structured (sparse) matrices. This allows the user to work with the 
covariance as they typically would with a concrete matrix, but under the hood linear algebra
operations are performed that exploit the structure in the matrix. We walk through a few 
different structured covariance operators below. 

### Diagonal Covariance

We start by considering a diagonal covariance matrix. In this case, we need only store `d` numbers, the 
variances forming the diagonal of the matrix. The full `(d, d)` matrix can always be formed by calling 
`to_dense()`, but this is rarely necessary nor recommended. Operations involving the diagonal linear 
operator (e.g., those necessary to sample from the Gaussian or compute its log density) will be carried 
out efficiently using only the diagonal entries.

In [19]:
cov_diag = DiagonalLinOp(np.array([1., 4., 16.]))

print(cov_diag)
print(cov_diag.shape)
print(cov_diag.dtype)
print(cov_diag.diagonal)
print(cov_diag.to_dense())

DiagonalLinOp(shape=(3, 3), dtype=float64)
(3, 3)
float64
[ 1.  4. 16.]
[[ 1.  0.  0.]
 [ 0.  4.  0.]
 [ 0.  0. 16.]]


In [21]:
x = Gaussian(np.zeros(3), cov_diag)
samp = x.sample(3)
dens = x.density(samp)


print("The covariance is represented by the diagonal linear operator:")
print(x.cov)

print("\nThe sampling method will exploit the sparse structure of the operator:")
print(samp)

print("\nAs will the density and other methods:")
print(dens)

The covariance is represented by the diagonal linear operator:
DiagonalLinOp(shape=(3, 3), dtype=float64)

The sampling method will exploit the sparse structure of the operator:
[[-8.50055362e-04 -8.03051675e-01  1.67306913e+00]
 [ 1.48557626e-01 -1.18413919e+00 -5.70969118e+00]
 [ 9.64893177e-01 -3.47766809e+00  7.18797988e-01]]

As will the density and other methods:
[0.00670875 0.00237839 0.0010812 ]


### Covariance Represented by Cholesky Factor

In algorithms involving Gaussian distributions, it is common to represent the Gaussian covariance 
$C$ indirectly via its Cholesky factor. If using the lower Cholesky factor this is a `(d, d)` lower
triangular matrix $L$ satisfying $C = LL^\top$. We can construct a Gaussian from its Cholesky factor
by passing the covariance as a `CholeskyLinOp` object. We first represent `L` itself as a `TriangularLinOp`
before feeding this to `CholeskyLinOp`.

We'll also construct a Gaussian with a dense covariance operator for comparison.

In [33]:
# Mean and covariance of Gaussian
m = np.array([0, 1])
C = np.array([[1.0, 0.8], [0.8, 1.0]])

# Dense representation
x_dense = Gaussian(m, C)

# Cholesky factor
L = np.linalg.cholesky(C, upper=False)
print("The Cholesky factor:")
print(L)

# Cholesky representation of covariance
L = TriangularLinOp(L, lower=True)
C_from_L = CholeskyLinOp(L)
x_chol = Gaussian(m, C_from_L)


The Cholesky factor:
[[1.  0. ]
 [0.8 0.6]]


In [35]:
# Investigating the covariance operator

print("Covariance operator:")
print(x_chol.cov)

print("\nThe Cholesky root (also represented as a linear operator):")
print(x_chol.cov.root)

print("\nDense Cholesky factor:")
print(x_chol.cov.root.to_dense())

print("\nDense covariance matrix:")
print(x_chol.cov.to_dense())

Covariance operator:
CholeskyLinOp(shape=(2, 2), dtype=float64)

The Cholesky root (also represented as a linear operator):
TriangularLinOp(shape=(2, 2), dtype=float64)

Dense Cholesky factor:
[[1.  0. ]
 [0.8 0.6]]

Dense covariance matrix:
[[1.  0.8]
 [0.8 1. ]]


In [36]:
# Testing that the two representations are equivalent
samp = x_chol.sample(10)

print("Log density evaluations of x_chol and x_dense are equal:")
np.array_equal(x_chol.log_density(samp),
               x_dense.log_density(samp))

AttributeError: 'numpy.ndarray' object has no attribute 'matmat'